# 02 - Revenue Modeling

Loads cleaned encounters, computes:
- fiscal year per encounter
- month bucket
- units (time-based CPTs)
- revenue using FY rate tables

Outputs monthly rollups:
- total revenue by month
- revenue by facility by month

In [7]:
import sys
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.revenue_model import add_units_and_revenue, rollups  # noqa: E402

## Load cleaned file

In [8]:
CLEAN_PATH = ROOT / "data" / "clean_encounters.csv"
df = pd.read_csv(CLEAN_PATH)

print("Rows:", len(df))
df.head()

Rows: 12327


,sex,facility,encounter_type,encounter_date,race,patient_status,cpt_code,duration_min,is_billable,encounter_status
0,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,African American,X Discharged,H0006,30.0,Yes,Closed
1,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,White,X Discharged,H0006,30.0,Yes,Closed
2,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,White,Active,H0006,30.0,Yes,Closed
3,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,White,X Discharged,H0006,30.0,Yes,Closed
4,Female,Women's Safe Harbor,Safe Harbor PRC Individual - MDHHS,2023-04-03,White,X Discharged,H0038,15.0,Yes,Closed


## Compute encounter-level revenue

In [9]:
df["encounter_date"] = pd.to_datetime(df["encounter_date"], errors="coerce")
df = df.dropna(subset=["encounter_date"])

enc = add_units_and_revenue(df)

enc.head()

,sex,facility,encounter_type,encounter_date,race,patient_status,cpt_code,duration_min,is_billable,encounter_status,fy,month,units,rate,revenue
0,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,African American,X Discharged,H0006,30.0,Yes,Closed,FY23,2023-04-01,1,45.5,45.5
1,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,White,X Discharged,H0006,30.0,Yes,Closed,FY23,2023-04-01,1,45.5,45.5
2,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,White,Active,H0006,30.0,Yes,Closed,FY23,2023-04-01,1,45.5,45.5
3,Male,Men's Safe Harbor,Case Management 30 Min,2023-04-03,White,X Discharged,H0006,30.0,Yes,Closed,FY23,2023-04-01,1,45.5,45.5
4,Female,Women's Safe Harbor,Safe Harbor PRC Individual - MDHHS,2023-04-03,White,X Discharged,H0038,15.0,Yes,Closed,FY23,2023-04-01,1,24.0,24.0


## Monthly rollups (total + facility)

In [10]:
monthly_total, monthly_by_facility, monthly_by_code = rollups(enc)

monthly_total.head(), monthly_by_facility.head(), monthly_by_code.head()

(       month  encounters  client_hours  total_units  revenue
 0 2022-10-01         246    179.383333          516  16315.5
 1 2022-11-01         319    191.250000          550  19085.5
 2 2022-12-01         330    137.816667          420  17170.5
 3 2023-01-01         350    121.033333          374  17243.0
 4 2023-02-01         305    118.066667          334  15043.5,
        month             facility  encounters  total_units  revenue
 0 2022-10-01    Men's Safe Harbor         121          198   6758.5
 1 2022-10-01           Outpatient          18           25   1982.5
 2 2022-10-01  Women's Safe Harbor         107          293   7574.5
 3 2022-11-01    Men's Safe Harbor         181          184   8666.5
 4 2022-11-01           Outpatient          14           14   1271.0,
        month cpt_code  encounters  total_units  revenue
 0 2022-10-01    90832           6            6    390.0
 1 2022-10-01    90837          11           11   1419.0
 2 2022-10-01    H0001           1       

## Save outputs

In [11]:
OUT_TOTAL = ROOT / "outputs" / "monthly_total.csv"
OUT_FAC = ROOT / "outputs" / "monthly_by_facility.csv"
OUT_CODE = ROOT / "outputs" / "monthly_by_code.csv"
OUT_ENC = ROOT / "outputs" / "encounters_with_revenue.csv"

monthly_total.to_csv(OUT_TOTAL, index=False)
monthly_by_facility.to_csv(OUT_FAC, index=False)
monthly_by_code.to_csv(OUT_CODE, index=False)
enc.to_csv(OUT_ENC, index=False)

print("Wrote:", OUT_TOTAL)
print("Wrote:", OUT_FAC)
print("Wrote:", OUT_CODE)
print("Wrote:", OUT_ENC)

Wrote: /mnt/c/Users/glopp/Python Projects/multi-year-revenue-trends/outputs/monthly_total.csv
Wrote: /mnt/c/Users/glopp/Python Projects/multi-year-revenue-trends/outputs/monthly_by_facility.csv
Wrote: /mnt/c/Users/glopp/Python Projects/multi-year-revenue-trends/outputs/monthly_by_code.csv
Wrote: /mnt/c/Users/glopp/Python Projects/multi-year-revenue-trends/outputs/encounters_with_revenue.csv
